In [1]:
!pip install deepface tensorflow onnx tf2onnx opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=96d78374f14ff9fa0872241c831e84f2e8a8c118e1e4f7d2cf7e8aa56fc77403
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [2]:
! rm -rf /kaggle/working/face_embedding_vgg_face.onnx /kaggle/working/state.db /kaggle/working/user_2d_embeddings.bin

In [3]:
import os
import cv2
import numpy as np
from deepface import DeepFace
import struct
import shutil

# --- Configuration ---
# MODIFIEZ CECI: Doit correspondre au nom de votre dataset sur Kaggle
# Ce dossier doit contenir directement les sous-dossiers de chaque utilisateur
# Exemple: si votre dataset s'appelle 'kinect-face-data' et contient des dossiers 'UserA', 'UserB',
# alors KAGGLE_DATASET_ROOT_DIR = "/kaggle/input/kinect-face-data/"
KAGGLE_DATASET_ROOT_DIR = "/kaggle/input/projet-dataset-1/" # <--- VÉRIFIEZ ET ADAPTEZ CECI SOIGNEUSEMENT

KAGGLE_OUTPUT_DIR = "/kaggle/working/"
OUTPUT_EMBEDDINGS_BIN = os.path.join(KAGGLE_OUTPUT_DIR, "user_2d_embeddings.bin")

MODEL_NAME = "VGG-Face" 
DETECTOR_BACKEND = 'opencv'

user_reference_embeddings = {}

print(f"--- Génération des Embeddings de Référence (Format Binaire) ---")
print(f"Utilisation du modèle : {MODEL_NAME} avec le backend de détection : {DETECTOR_BACKEND}")
print(f"Lecture des dossiers utilisateurs depuis : {KAGGLE_DATASET_ROOT_DIR}")
print(f"Sauvegarde des embeddings dans : {OUTPUT_EMBEDDINGS_BIN}")

if not os.path.exists(KAGGLE_DATASET_ROOT_DIR):
    print(f"ERREUR: Le dossier racine du dataset '{KAGGLE_DATASET_ROOT_DIR}' n'a pas été trouvé.")
    print("Vérifiez le chemin KAGGLE_DATASET_ROOT_DIR et assurez-vous que votre dataset est correctement ajouté au notebook.")
    exit()

if not os.listdir(KAGGLE_DATASET_ROOT_DIR):
    print(f"ERREUR: Le dossier racine du dataset '{KAGGLE_DATASET_ROOT_DIR}' est vide.")
    exit()

# Parcourir les dossiers de chaque utilisateur
found_users = False
for user_name in os.listdir(KAGGLE_DATASET_ROOT_DIR):
    user_dir_path = os.path.join(KAGGLE_DATASET_ROOT_DIR, user_name)
    if os.path.isdir(user_dir_path):
        found_users = True
        print(f"\nTraitement des images pour l'utilisateur : {user_name} (depuis {user_dir_path})")
        user_embeddings_list = []
        
        image_files = [f for f in os.listdir(user_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not image_files:
            print(f"  Aucune image trouvée dans {user_dir_path}")
            continue
        
        print(f"  {len(image_files)} images trouvées pour {user_name}.")
        processed_image_count = 0
        for image_file in image_files:
            image_path = os.path.join(user_dir_path, image_file)
            try:
                embedding_objs = DeepFace.represent(
                    img_path=image_path,
                    model_name=MODEL_NAME,
                    enforce_detection=True, 
                    detector_backend=DETECTOR_BACKEND,
                    align=True 
                )
                
                if embedding_objs and len(embedding_objs) > 0:
                    embedding = embedding_objs[0]['embedding'] 
                    if embedding and isinstance(embedding, list) and len(embedding) > 0: 
                        user_embeddings_list.append(embedding)
                        processed_image_count += 1
                    # else:
                    #     print(f"    Visage non détecté ou embedding vide pour {image_file}.") # Peut être trop verbeux
                # else:
                #     print(f"    DeepFace.represent a retourné une liste vide pour {image_file}.") # Peut être trop verbeux

            except ValueError as ve: # Souvent levé si aucun visage n'est détecté avec enforce_detection=True
                 print(f"    AVERTISSEMENT pour {image_file}: {ve} (Probablement aucun visage détecté ou problème d'alignement)")
            except Exception as e:
                print(f"    ERREUR lors du traitement de {image_file}: {type(e).__name__} - {e}")
        
        if user_embeddings_list:
            mean_embedding = np.mean(np.array(user_embeddings_list, dtype=np.float32), axis=0)
            user_reference_embeddings[user_name] = mean_embedding.tolist()
            print(f"  Embedding de référence moyen calculé pour {user_name} à partir de {len(user_embeddings_list)} images (taille: {len(mean_embedding)})")
        else:
            print(f"  Aucun embedding valide n'a pu être généré pour {user_name} à partir des {processed_image_count}/{len(image_files)} images traitées.")
    # else:
    #     print(f"  '{user_name}' n'est pas un dossier. Ignoré.")

if not found_users:
    print(f"ERREUR: Aucun dossier utilisateur trouvé dans '{KAGGLE_DATASET_ROOT_DIR}'. Veuillez vérifier la structure de votre dataset.")
    print(f"Contenu de '{KAGGLE_DATASET_ROOT_DIR}': {os.listdir(KAGGLE_DATASET_ROOT_DIR)}")


try:
    with open(OUTPUT_EMBEDDINGS_BIN, 'wb') as f_bin:
        print(f"\nSauvegarde des embeddings de référence dans '{OUTPUT_EMBEDDINGS_BIN}'...")
        if not user_reference_embeddings:
            print("Aucun embedding à sauvegarder.")
            f_bin.write(struct.pack('I', 0))
        else:
            f_bin.write(struct.pack('I', len(user_reference_embeddings)))
            for user_name, embedding_list_float in user_reference_embeddings.items():
                name_bytes = user_name.encode('utf-8')
                f_bin.write(struct.pack('I', len(name_bytes)))
                f_bin.write(name_bytes)
                embedding_size = len(embedding_list_float)
                f_bin.write(struct.pack('I', embedding_size))
                for val_float in embedding_list_float:
                    f_bin.write(struct.pack('f', val_float))
                print(f"  Embedding binaire sauvegardé pour {user_name}")
    print("Sauvegarde binaire des embeddings terminée.")
except IOError as e:
    print(f"ERREUR: Impossible d'écrire dans le fichier binaire '{OUTPUT_EMBEDDINGS_BIN}': {e}")
except Exception as e_gen:
    print(f"ERREUR inattendue lors de la sauvegarde binaire: {e_gen}")

print(f"\n--- Fin de la génération des embeddings ---")
print(f"Le fichier binaire des embeddings devrait se trouver dans : {KAGGLE_OUTPUT_DIR}")


2025-06-26 08:18:24.202144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750925904.376955      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750925904.427921      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


25-06-26 08:18:35 - Directory /root/.deepface has been created
25-06-26 08:18:35 - Directory /root/.deepface/weights has been created
--- Génération des Embeddings de Référence (Format Binaire) ---
Utilisation du modèle : VGG-Face avec le backend de détection : opencv
Lecture des dossiers utilisateurs depuis : /kaggle/input/projet-dataset-1/
Sauvegarde des embeddings dans : /kaggle/working/user_2d_embeddings.bin

Traitement des images pour l'utilisateur : Samuel (depuis /kaggle/input/projet-dataset-1/Samuel)
  166 images trouvées pour Samuel.


I0000 00:00:1750925916.425972      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


25-06-26 08:18:36 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:13<00:00, 41.8MB/s] 
I0000 00:00:1750925935.883506      35 cuda_dnn.cc:529] Loaded cuDNN version 90300


    AVERTISSEMENT pour Samuel_face_285.png: Face could not be detected in /kaggle/input/projet-dataset-1/Samuel/Samuel_face_285.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. (Probablement aucun visage détecté ou problème d'alignement)
    AVERTISSEMENT pour Samuel_face_108.png: Face could not be detected in /kaggle/input/projet-dataset-1/Samuel/Samuel_face_108.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. (Probablement aucun visage détecté ou problème d'alignement)
    AVERTISSEMENT pour Samuel_face_072.png: Face could not be detected in /kaggle/input/projet-dataset-1/Samuel/Samuel_face_072.png.Please confirm that the picture is a face photo or consider to set enforce_detection param to False. (Probablement aucun visage détecté ou problème d'alignement)
    AVERTISSEMENT pour Samuel_face_082.png: Face could not be detected in /kaggle/input/projet-dataset-1/Samuel/Samuel

In [4]:
import tensorflow as tf
import tf2onnx
import os
from deepface import DeepFace 

# --- Configuration ---
MODEL_NAME_TO_CONVERT = "VGG-Face" 
INPUT_HEIGHT = 224 
INPUT_WIDTH = 224
INPUT_CHANNELS = 3  
ONNX_INPUT_NAME = "input_1" # Nom d'entrée typique pour les modèles Keras.

KAGGLE_OUTPUT_DIR = "/kaggle/working/"
ONNX_MODEL_FILENAME = f"face_embedding_{MODEL_NAME_TO_CONVERT.lower().replace('-', '_')}.onnx"
ONNX_MODEL_OUTPUT_PATH = os.path.join(KAGGLE_OUTPUT_DIR, ONNX_MODEL_FILENAME)

print(f"--- Conversion du Modèle en ONNX ---")
print(f"Modèle à convertir : {MODEL_NAME_TO_CONVERT}")
print(f"Dimensions d'entrée attendues (H, W, C) : ({INPUT_HEIGHT}, {INPUT_WIDTH}, {INPUT_CHANNELS})")
print(f"Nom d'entrée ONNX attendu pour TensorSpec : {ONNX_INPUT_NAME}")
print(f"Sortie ONNX prévue : {ONNX_MODEL_OUTPUT_PATH}")

keras_model_to_convert = None
try:
    print(f"Construction (et téléchargement si nécessaire) du modèle '{MODEL_NAME_TO_CONVERT}' par DeepFace...")
    # DeepFace.build_model est la fonction principale pour obtenir les modèles.
    # Elle devrait retourner un objet modèle Keras ou un wrapper.
    loaded_object = DeepFace.build_model(MODEL_NAME_TO_CONVERT)
    print(f"Objet chargé par DeepFace.build_model(). Type: {type(loaded_object)}")

    # Vérifier si l'objet chargé est directement un modèle Keras
    if isinstance(loaded_object, tf.keras.Model):
        print(f"L'objet chargé est directement une instance de tf.keras.Model.")
        keras_model_to_convert = loaded_object
    # Si ce n'est pas un modèle Keras direct, vérifier s'il a un attribut '.model'
    # C'est une heuristique courante pour les wrappers de modèles.
    elif hasattr(loaded_object, 'model') and isinstance(loaded_object.model, tf.keras.Model):
        print(f"L'objet chargé a un attribut '.model' qui est une instance de tf.keras.Model. Utilisation de loaded_object.model.")
        keras_model_to_convert = loaded_object.model
    else:
        print(f"ERREUR: L'objet chargé par DeepFace.build_model() pour {MODEL_NAME_TO_CONVERT} n'est pas un tf.keras.Model directement,")
        print(f"et n'a pas d'attribut '.model' attendu. Type de l'objet: {type(loaded_object)}")
        print("La conversion en ONNX ne peut pas continuer sans un modèle Keras valide.")
        exit()
        
    if keras_model_to_convert is None:
        print(f"ERREUR: Impossible d'obtenir un modèle Keras valide pour {MODEL_NAME_TO_CONVERT}")
        exit()
        
    print(f"Modèle Keras à convertir (type: {type(keras_model_to_convert)}):")
    keras_model_to_convert.summary()

except Exception as e:
    print(f"ERREUR lors de la construction/chargement du modèle '{MODEL_NAME_TO_CONVERT}': {type(e).__name__} - {e}")
    exit()


# Définir la signature d'entrée pour la conversion ONNX
input_signature = (tf.TensorSpec((None, INPUT_HEIGHT, INPUT_WIDTH, INPUT_CHANNELS), tf.float32, name=ONNX_INPUT_NAME),)
print(f"Utilisation de la signature d'entrée pour ONNX: {input_signature}")

# Convertir le modèle Keras en ONNX
try:
    print(f"\nConversion du modèle Keras en ONNX (opset 13)...")
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(
        keras_model_to_convert, 
        input_signature=input_signature,
        opset=13, 
        output_path=ONNX_MODEL_OUTPUT_PATH
    )
    print("Conversion en ONNX terminée.")
    print(f"Modèle ONNX sauvegardé ici : {ONNX_MODEL_OUTPUT_PATH}")
    print("\nVous pouvez maintenant télécharger ce fichier .onnx depuis l'onglet 'Output' de ce notebook Kaggle.")
    print(f"Utilisez un outil comme Netron (https://netron.app/) pour inspecter le modèle .onnx.")
    print(f"Vérifiez que le nom d'entrée dans Netron correspond bien à '{ONNX_INPUT_NAME}'.")

except Exception as e:
    print(f"ERREUR lors de la conversion du modèle Keras en ONNX : {type(e).__name__} - {e}")
    print(f"Vérifiez que les dimensions d'entrée ({INPUT_HEIGHT}, {INPUT_WIDTH}, {INPUT_CHANNELS}) correspondent au modèle {MODEL_NAME_TO_CONVERT}.")
    print(f"Assurez-vous que '{ONNX_INPUT_NAME}' est un nom d'entrée valide pour le modèle Keras.")
    print(f"Essayez un autre opset si nécessaire (ex: opset=11 ou opset=12).")

print(f"\n--- Fin de la conversion ONNX ---")


--- Conversion du Modèle en ONNX ---
Modèle à convertir : VGG-Face
Dimensions d'entrée attendues (H, W, C) : (224, 224, 3)
Nom d'entrée ONNX attendu pour TensorSpec : input_1
Sortie ONNX prévue : /kaggle/working/face_embedding_vgg_face.onnx
Construction (et téléchargement si nécessaire) du modèle 'VGG-Face' par DeepFace...
Objet chargé par DeepFace.build_model(). Type: <class 'deepface.models.facial_recognition.VGGFace.VggFaceClient'>
L'objet chargé a un attribut '.model' qui est une instance de tf.keras.Model. Utilisation de loaded_object.model.
Modèle Keras à convertir (type: <class 'tf_keras.src.engine.functional.Functional'>):
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_input (Inpu  [(None, 224, 224, 3)]     0         
 tLayer)                                                         
                                                                 
 zero_padding2d 

I0000 00:00:1750926696.475176      35 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1750926696.475340      35 single_machine.cc:361] Starting new session
I0000 00:00:1750926696.476438      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1750926719.242536      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1750926722.286480      35 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1750926722.286692      35 single_machine.cc:361] Starting new session
I0000 00:00:1750926722.287846      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 wi

Conversion en ONNX terminée.
Modèle ONNX sauvegardé ici : /kaggle/working/face_embedding_vgg_face.onnx

Vous pouvez maintenant télécharger ce fichier .onnx depuis l'onglet 'Output' de ce notebook Kaggle.
Utilisez un outil comme Netron (https://netron.app/) pour inspecter le modèle .onnx.
Vérifiez que le nom d'entrée dans Netron correspond bien à 'input_1'.

--- Fin de la conversion ONNX ---
